# BAIS:3250 - Data Wrangling
## Descriptive Statistics
___ 


Libraries used:

- _**pandas**_ for dataframes and data wrangling functions
- _**matplotlib**_ for creating simple plots when performing exploratory data analysis
- _**seaborn**_ is a high-level interface for creating visually appealing and informative statistical graphics
- _**scipy.stats**_ are tools for both basic and advanced statistical operations


In [ ]:
from IPython.html.services.config import ConfigManager

ip = get_ipython()
cm = ConfigManager(parent=ip)
cm.update("notebook", {"ruler_column": [80]})

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from IPython.display import display
from scipy.stats import chi2_contingency, pearsonr

### Read data into a pandas DataFrame

In [ ]:
# read in pipe delimited file named mortgage.txt
mortgage_df = pd.read_csv("mortgage.txt", encoding="utf-8", sep="|")

display(mortgage_df)

### Statistics for One Continuous Variable

In [ ]:
print(mortgage_df["CreditScore"].unique())
print(mortgage_df["CreditScore"].isna().sum())

In [ ]:
mortgage_df.dtypes

<font color="darkmagenta">CreditScore is a numeric data type.</font>

<br>

#### .describe() function

The ```.describe()``` function will automatically calculate many statistics for you with a single function.

In [ ]:
# use the .describe() function
descriptive_stats = mortgage_df["CreditScore"].describe()

# descriptive statistics
print(descriptive_stats)

In [ ]:
print(mortgage_df.describe())

<br>

#### Manually calculate descriptive statistics

In [ ]:
# define the dataframe and column we are studying
column = mortgage_df["CreditScore"]
column_name = "CreditScore"

In [ ]:
print(f"Statistics of the {column_name} column")
print("-" * 50)

print(f"{column_name} count: {column.count()}")  # number of non-null entries
print(f"{column_name} mean: {column.mean()}")  # sum of all values / n


# spread describes the extremes and variation
print(f"{column_name} min: {column.min()}")
print(f"{column_name} 25%: {column.quantile(0.25)}")  # 25th percentile
print(
    f"{column_name} median (50%): {column.median()}"
)  # 50th percentile (same as 2nd quartile)
print(f"{column_name} 75%: {column.quantile(0.75)}")  # 75th percentile
print(f"{column_name} max: {column.max()}")
print(
    f"{column_name} interquartile range (IQR): {column.quantile(0.75) - column.quantile(0.25)}"
)
print(f"{column_name} range: {column.max() - column.min()}")

# the most frequent value(s)
# - if multiple modes exist, it returns all of them as a Series
modes = column.mode()
# handle cases where there are multiple modes
if len(modes) > 1:
    print(f"{column_name} modes: {list(modes)}")
else:
    print(f"{column_name} mode:  {modes[0]}")

print("-" * 10)

# average of squared difference between each data point and the mean
print(f"{column_name} variance: {column.var()}")
print(f"{column_name} std_dev: {column.std()}")  # square root of variance


print(f"{column_name} skewness: {column.skew()}")


print(f"{column_name} kurtosis: {column.kurt()}")

<br>

#### Shape can be confirmed with visualization. Let's use _Seaborn_.

##### Boxplot

In [ ]:
# set plot size
plt.figure(figsize=(8, 6))

# create a box plot for 'CreditScore'
sns.boxplot(x=column, color="skyblue")

# add title and labels
plt.title("Box Plot of Credit Scores", fontsize=16)
plt.xlabel("Credit Score", fontsize=12)

# display the plot
plt.show()

<font color="darkmagenta">Shape is symmetrical (not skewed), with few outliers.</font>

##### Histogram.

In [ ]:
# set plot size
plt.figure(figsize=(8, 6))

# create a histogram for 'CreditScore'
sns.histplot(column, bins=20, kde=True)

# add title and labels
plt.title("Histogram of Credit Scores", fontsize=16)
plt.xlabel("Credit Score", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# display the plot
plt.show()

<font color="darkmagenta">Shape is symmetrical, and unimodal (one peak).  
Somewhat bell-shaped (might be normally distributed).</font>  

##### Check normality with a Q-Q plot.  
A Quantile-Quantile plot compares actual quantiles to expected values under normal distribution 

```scipy.stats.probplot()``` creates the Q-Q plot.  
```dist="norm"``` specifies the theoretical distribution (normal) to compare against.  
```plot=plt``` directs the plot to matplotlib for rendering.

In [ ]:
# create a Q-Q plot for 'CreditScore'
stats.probplot(column, dist="norm", plot=plt)

# add title
plt.title("Q-Q Plot of Credit Scores")

# display the plot
plt.show()

<font color="darkmagenta">Marks stay close to the red line, suggesting that credit scores in the data are very close to a normal distribution.</font>

<br>

#### Shapiro-Wilk test
The Shapiro test checks if a dataset follows a normal distribution.  
**Null Hypothesis (H<sub>0</sub>)**: The data is normally distributed.  
**Alternative Hypothesis (H<sub>1</sub>)**: The data is not normally distributed.

In [ ]:
# Shapiro-Wilk test for normality
stat, p_value = stats.shapiro(column)

# display the results
print(f"Shapiro-Wilk Test Statistic: {stat}")
print(f"P-Value: {p_value}")

# interpretation
alpha = 0.05  # Significance level
if p_value > alpha:
    print("The data follows a normal distribution (fail to reject H0).")
else:
    print("The data does not follow a normal distribution (reject H0).")

<font color="darkmagenta">Interpretation:  
If **p-value > 0.05**, the data is normally distributed (fail to reject the null hypothesis H<sub>0</sub>).  
If **p-value ≤ 0.05**, the data is not normally distributed (reject H<sub>0</sub>). </font>  

<br>

### Statistics for One Categorical Variable

#### Make the column a _category_.

In [ ]:
print(mortgage_df["Default"].unique())

In [ ]:
# change the column to a categorical data type
mortgage_df["Default"] = mortgage_df["Default"].astype("category")

In [ ]:
# verify it is a categorical data type
print(mortgage_df["Default"].dtypes)

In [ ]:
# rename the categorical column values with meaningful labels
mortgage_df["Default"] = mortgage_df["Default"].cat.rename_categories(
    {1: "Defaulted", 0: "Not Defaulted"}
)

print(mortgage_df.head())

#### Frequency counts

In [ ]:
# define the dataframe and column we are studying
column = mortgage_df["Default"]
column_name = "Default"

In [ ]:
# frequency counts
default_counts = column.value_counts()
print("Frequency Counts:")
print(default_counts)

_Notice this is a Series and the row index are the values (Defaulted / Not Defaulted)._

<br>

Frequency counts sorted in descending order by **level**

In [ ]:
# frequency counts sorted in descending order by the value
default_counts = column.value_counts().sort_index(ascending=False)
print("Frequency Counts (Sorted by Value):")
print(default_counts)

Frequency counts sorted in descending order by **count value**

In [ ]:
# frequency counts sorted in descending order by the count
default_counts_sorted = column.value_counts().sort_values(ascending=False)
print("\nFrequency Counts (Sorted by Frequency):")
print(default_counts_sorted)

<br>

#### Proportions table

A _proportions table_ is used to express counts as percentages or proportions of the total. It helps to understand the distribution of categorical data.

In [ ]:
# frequency counts
default_counts = column.value_counts()

# proportions table based on frequency counts
proportions_table = default_counts / default_counts.sum()

# Display the proportions table
print("Proportions Table:")
print(proportions_table)

<br>

#### Visualizations

##### Create a simple bar chart with Seaborn

In [ ]:
# create a bar chart
plt.figure(figsize=(8, 6))
sns.barplot(x=default_counts.index, y=default_counts.values)

# add title and labels
plt.title("Frequency of Mortgage Defaults", fontsize=16)
plt.xlabel("Default (0 = Not Defaulted, 1 = Defaulted)", fontsize=12)
plt.ylabel("Count", fontsize=12)

# display the plot
plt.show()

<br>

##### Create a _fancier_ bar chart with Seaborn

In [ ]:
# convert 'Default' to string to use it as a hue
mortgage_df["Default"] = mortgage_df["Default"].astype(str)

# frequency counts of 'Default'
default_counts = mortgage_df["Default"].value_counts()

# create a bar chart with different colors using hue
plt.figure(figsize=(8, 6))
ax = sns.barplot(
    x=default_counts.index,
    y=default_counts.values,
    hue=default_counts.index,
    palette=["skyblue", "salmon"],
    dodge=False,
    legend=False,  # suppress the legend
)

# add title and labels
plt.title("Frequency of Mortgage Defaults", fontsize=16)
# plt.xlabel("Default (0 = Not Defaulted, 1 = Defaulted)", fontsize=12)
plt.ylabel("Count", fontsize=12)

# annotate each bar with its value
for container in ax.containers:
    ax.bar_label(container, label_type="edge", fontsize=12)

# display the plot
plt.show()

##### Create a horizontal bar chart

In [ ]:
# convert 'Default' to string to use it as a hue
mortgage_df["Default"] = mortgage_df["Default"].astype(str)

# frequency counts of 'Default'
default_counts = mortgage_df["Default"].value_counts()

# create a horizontal bar chart with different colors using hue
plt.figure(figsize=(8, 6))
ax = sns.barplot(
    y=default_counts.index,
    x=default_counts.values,
    hue=default_counts.index,
    palette=["skyblue", "salmon"],
    dodge=False,
    legend=False,  # Suppress the legend
)

# add title and labels
plt.title("Frequency of Mortgage Defaults", fontsize=16)
# plt.ylabel("Default (0 = Not Defaulted, 1 = Defaulted)", fontsize=12)
plt.xlabel("Count", fontsize=12)

# annotate each bar with its value
for container in ax.containers:
    ax.bar_label(container, label_type="edge", fontsize=12)

# dDisplay the plot
plt.show()

##### Create a pie chart

In [ ]:
# Frequency counts of 'Default'
default_counts = mortgage_df["Default"].value_counts()

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(
    default_counts,
    labels=["Not Defaulted", "Defaulted"],
    autopct="%1.1f%%",
    startangle=90,
    colors=["skyblue", "salmon"],
    wedgeprops={"edgecolor": "black"},
)

# Add title
plt.title("Proportion of Mortgage Defaults", fontsize=16)

# Display the pie chart
plt.show()

_Note: Pie charts should not be used when there are many categories (rule of thumb: <= 5) or when categories overlap._

<br>

### Statistics for Two Categorical Variables 

Prepare the data

In [ ]:
mortgage_df = pd.read_csv("mortgage.txt", encoding="utf-8", sep="|")
display(mortgage_df)

In [ ]:
mortgage_df.isna().sum()

In [ ]:
print(mortgage_df["Default"].unique())
print(mortgage_df["Year"].unique())

In [ ]:
print(mortgage_df[mortgage_df["Year"] == "-"])

# count the number of rows where 'Year' is '-'
print(mortgage_df[mortgage_df["Year"] == "-"].shape[0])

In [ ]:
print(mortgage_df.shape)
# remove rows where 'Year' is '-'
mortgage_df = mortgage_df[mortgage_df["Year"] != "-"].reset_index(drop=True)
print("row dropped")
print(mortgage_df.shape)

In [ ]:
# change the column to a categorical data type
mortgage_df["Default"] = mortgage_df["Default"].astype("category")
mortgage_df["Year"] = mortgage_df["Year"].astype("category")

# verify it is a categorical data type
print(mortgage_df["Default"].dtypes)
print(mortgage_df["Year"].dtypes)

<br>

##### Compare frequency and proportion across groups

In [ ]:
# create a frequency table for 'Default' and 'Year'
frequency_table = pd.crosstab(mortgage_df["Default"], mortgage_df["Year"])

# display the frequency table
print("Frequency Table:")
print(frequency_table)

```.crosstab()``` is used to create a contingency table, which summarizes the relationship between two or more categorical variables. This table is a bit confusing. Notice the difference between the row index of 0 and 1 (far left) and the value (1 (defaulted) & 0 (not defaulted).

<font color="darkmagenta">Interpretation: Number of non-defaults (0) stays fairly consistent from year to year, but more defaults increase in 2008 and 2009.</font>

<br>

##### Visualize the two categories with a stacked bar chart

In [ ]:
# prepare the data: Melt the contingency table to long format
default_year_table = pd.crosstab(
    mortgage_df["Year"], mortgage_df["Default"]
).reset_index()
default_year_table = pd.melt(
    default_year_table, id_vars=["Year"], var_name="Default", value_name="Count"
)

# convert 'Default' column to categorical with proper labels
default_year_table["Default"] = (
    default_year_table["Default"].astype(int).map({0: "Non-Default", 1: "Default"})
)

# create the bar chart using Seaborn
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=default_year_table,
    x="Year",
    y="Count",
    hue="Default",
    palette=["skyblue", "salmon"],
)

# annotate each bar with its value
for container in ax.containers:
    ax.bar_label(container, fmt="%d", label_type="edge", fontsize=12)
    # Annotate each bar with its value at the top of the bar

# add title and labels
plt.title("Number of Defaults vs. Non-Defaults per Year", fontsize=16)
plt.xlabel("Year", fontsize=12)
plt.ylabel("Count", fontsize=12)

# adjust the legend
plt.legend(
    title="Status",
    loc="upper center",
    bbox_to_anchor=(0.5, -0.1),
    ncol=2,
    frameon=False,
)

# display the plot
plt.show()

<font color="darkmagenta">It looks like the number of non-defaults has remained steady, but the number of defaults seems to be increasing across the years.</font>

<br>

##### Joint Probablility Distribution  
Total of entire table = 100%  

In [ ]:
# Create a contingency table (similar to table() in R)
contingency_table = pd.crosstab(mortgage_df["Default"], mortgage_df["Year"])

# Calculate the joint probability distribution
joint_probability = round(contingency_table / contingency_table.values.sum(), 2)

# Display the joint probability distribution
print("Joint Probability Distribution:")
print(joint_probability)

<br>

##### Marginal probability  
Compare by row (row-wise)  

In [ ]:
# create a contingency table for 'Default' and 'Year'
contingency_table = pd.crosstab(mortgage_df["Default"], mortgage_df["Year"])

In [ ]:
# calculate marginal probabilities by row (axis=1)
marginal_probability_by_row = round(
    contingency_table.div(contingency_table.sum(axis=1), axis=0), 2
)

# display the marginal probability distribution (by row)
print("Marginal Probability Distribution (by row):")
print(marginal_probability_by_row)

<font color="darkmagenta">Interpretation: Majority of defaults occurred in 2008 and 2009, but that non-defaults are distributed evenly over the 3-year span.</font>

<br>

##### Marginal probability  
Column-wise  

In [ ]:
# calculate marginal probabilities by column (axis=0)
marginal_probability_by_column = round(
    contingency_table.div(contingency_table.sum(axis=0), axis=1), 2
)

# display the marginal probability distribution (by column)
print("\nMarginal Probability Distribution (by column):")
print(marginal_probability_by_column)

<font color="darkmagenta">Interpretation: Rate of default increased from 2007 (13% of mortgages) to 2008 (23%) to 2009 (30%)</font>

<br>

<font color="darkmagenta">But are the differences in default rates significant?  </font>  

<br>

#### Chi-Square Test of Independence
  
Tests if there is an association between two categorical variables. This test evaluates how likely it is that any observed differences arose purely by chance.  
  
Hypotheses:  
Null Hypothesis (H<sub>0</sub>): The two variables are independent (no association) if _p-value > 0.05_.  
Alternative Hypothesis (H<sub>1</sub>): The two variables are dependent (associated) if _p-value <= 0.05_.  

In [ ]:
# create a contingency table for 'Default' and 'Year'
contingency_table = pd.crosstab(mortgage_df["Default"], mortgage_df["Year"])

# perform the chi-square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# display the p-value and interpretation
print(f"Chi-Square Test p-value: {p_value:.4f}")

# set the significance level (alpha)
alpha = 0.05

# interpret the result
if p_value <= alpha:
    print("Reject H0: The number of defaults vs. non-defaults is related to the year.")
else:
    print(
        "Fail to reject H0: The number of defaults vs. non-defaults is independent of the year."
    )

<font color="darkmagenta">Interpretation: p is much smaller than 0.05, thus we reject the null hypothesis that _Year_ and _Default_ are independent. _Year_ and _Default_ are dependent.</font>

<br>

### Statistics for Two Continuous Variables

#### Pearson Correlation (Linear Correlation)

Correlation measures the **strength and direction of the linear relationship** between two continuous variables (e.g., height vs. weight).

It quantifies the degree to which two variables move together and is commonly denoted by the correlation coefficient _**r**_, which ranges from -1 to 1.
**_r_ = 1**: Perfect positive linear relationship (as one variable increases, the other also increases).
**_r_ = -1**: Perfect negative linear relationship (as one variable increases, the other decreases).
**_r_ = 0**: No linear relationship between the variables.

The Pearson correlation assumes that the data is continuous and normally distributed.

The Kendall or Spearman correlations may be more appropriate for comparing ordinal, rank data.


In [ ]:
# calculate Pearson correlation between 'CreditScore' and 'HomeAge'
pearson_corr = mortgage_df["CreditScore"].corr(mortgage_df["HomeAge"], method="pearson")

# display the result
print(f"Pearson Correlation: {pearson_corr:.4f}")

# interpretation
if abs(pearson_corr) < 0.3:
    print("Interpretation: No or weak linear correlation.")
elif 0.3 <= abs(pearson_corr) < 0.7:
    print("Interpretation: Moderate linear correlation.")
else:
    print("Interpretation: Strong linear correlation.")

<font color="darkmagenta">Interpretation: No linear correlation.</font>

<br>

#### Pearson Correlation Test

The Pearson correlation test is statistical hypothesis test that checks whether the linear relationship found in the Pearson correlation is statistically significant or if it could have occurred by chance. The test provides a _**p-value**_ to help make this decision.

Hypothesis test for significant correlation.   
H<sub>0</sub> = no correlation  
H<sub>1</sub> = correlation!=0  



In [ ]:
# perform the Pearson correlation test
corr_stat, p_value = pearsonr(mortgage_df["CreditScore"], mortgage_df["HomeAge"])

# display the correlation statistic and p-value
print(f"Pearson Correlation Coefficient: {corr_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# interpretation based on p-value
alpha = 0.05  # significance level

if p_value > alpha:
    print("Fail to reject H0: No significant correlation (correlation = 0).")
else:
    print("Reject H0: Significant correlation exists (correlation != 0).")

<font color="darkmagenta">Interpretation: p > 0.05, thus we fail to reject the null hypothesis that the correlation is 0. There is no correlation. </font>

<br>

##### Visualize relationships between continuous variables with **scatter plot**.

In [ ]:
# create the scatter plot with a regression line using Seaborn
plt.figure(figsize=(8, 6))
sns.regplot(
    x="CreditScore",
    y="HomeAge",
    data=mortgage_df,
    scatter_kws={"s": 50},  # adjust size of scatter points
    line_kws={"color": "red"},  # set line color
)

# add axis labels and title
plt.xlabel("Credit Score", fontsize=12)
plt.ylabel("Home Age", fontsize=12)
plt.title("Scatter Plot of Credit Score vs. Home Age", fontsize=16)

# display the plot
plt.show()

<font color="darkmagenta">Interpretation: no linear relationship.</font>